# Algorytmy macierzowe
## lab 2

In [1]:
import numpy as np
from time import time
from matplotlib import pyplot as plt

Własna klasa Number do zliczania ilości operacji

In [4]:
class Number(float):
    operation_counter = 0
    
    def __radd__(self, other):
        Number.operation_counter += 1
        return Number(super().__radd__(other))

    def __add__(self, other):
        Number.operation_counter += 1
        return Number(super().__add__(other))
    
    def __rsub__(self, other):
        Number.operation_counter += 1
        return Number(super().__rsub__(other))

    def __sub__(self, other):
        Number.operation_counter += 1
        return Number(super().__sub__(other))
    
    def __mul__ (self, other):
        Number.operation_counter += 1
        return Number(super().__mul__(other))
    
    def __rmul__ (self, other):
        Number.operation_counter += 1
        return Number(super().__rmul__(other))
    
    def __truediv__(self, other):
        Number.operation_counter += 1
        return Number(super().__truediv__(other))
    
    def __rtruediv__(self, other):
        Number.operation_counter += 1
        return Number(super().__rtruediv__(other))
    
    def counter_reset():
        Number.operation_counter = 0

Generowanie macierzy

In [2]:
def random_matrix(matrix_size, min_val, max_val):
    """Return matrix with random floats from [min_val, max_val)"""
    return (max_val - min_val) * np.random.random(matrix_size) + min_val

In [6]:
exp = 3
matrix_size = (2**exp, 2**exp)

min_val = 0.00000001
max_val = 1

A = random_matrix(matrix_size, min_val, max_val)
B = random_matrix(matrix_size, min_val, max_val)

### Rekurencyjne odwracanie macierzy

### Rekurencyjna LU faktoryzacja

### Rekurencyjne obliczanie wyznacznika